In [5]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.master("local[*]").appName("Fifa21_DataCleaning").getOrCreate()

KeyboardInterrupt: 

In [ ]:
fifa_df_v2 = spark.read.format("csv") \
    .option("header", "true") \
    .option("multiLine", "true") \
    .option("inferSchema","true") \
    .option("quote", "\"") \
    .option("escape", "\"") \
    .load("D:\Grapple\PySpark\DataSets\Fifa21\fifa raw data v2.csv")
display(fifa_df_v2)

In [ ]:
#Renaming the column Name

fifa_df_v2=fifa_df_v2.withColumnRenamed("↓OVA","OVA")

In [ ]:
# Modyfying the column Club Removing the "\n" from the column using regexp_replace
from pyspark.sql.functions import col, regexp_replace
fifa_df_v2=fifa_df_v2.withColumn("Club",regexp_replace(col("Club"),"\n",""))
fifa_df_v2.select("Club").show()

In [ ]:
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType, StringType
from pyspark.sql.functions import udf

# Replacing the column Height from String to integer
def replaceheight(x):
    x=x.strip()                             # removing the space using strip()
    if x is None:   
        return None
    if "'" in x:            
        feet, foot = x.split("'")           # assigning the value to feet and foot by spliting based on (')
        if "\"" in foot:
            foot=foot.replace("\"","")      # replacing the " with ""
        if foot=="":
            foot=0                          # if foot is empty then foot is 0
        
        return int(int(feet) * 30.48 + int(foot) * 2.54)    #converting  the value to cm
    elif "cm" in x:
        return int(x.replace("cm",""))      # replacing the cm with "" an tycasting to int 
    else:
        return None

# using userdefine function to replace the value
returnntype=udf(replaceheight,StringType())
# using with colummn to replace the value and casting the value to int
fifa_df_v2=fifa_df_v2.withColumn("Height", returnntype(col("Height"))).withColumn("Height",col("Height").cast("int"))

# Display how many types of data in the height column using regexp_replace, group by and count
display(fifa_df_v2.withColumn("Height",regexp_replace(col("Height"),"[0-9]","")).groupBy("Height").count().orderBy("count"))
display(fifa_df_v2.select("Height").dtypes)


In [ ]:
'''
# Method 2

from pyspark.sql.functions import col, when, split, regexp_replace

fifa_df_v2 = fifa_df_v2.withColumn(
    "Height",
    when(col("Height").like("%'%"),
         # Feet → split before '
         split(col("Height"), "'").getItem(0).cast("int") * 30.48 +
         # Inches → split after ' then remove all non-digits like " or ''
         regexp_replace(split(col("Height"), "'").getItem(1), r'[^0-9]', "").cast("int") * 2.54
    )
    .when(col("Height").like("%cm%"),
          regexp_replace(col("Height"), "cm", "").cast("int"))
    .otherwise(None)
).withColumn("Height", col("Height").cast("int"))

#fifa_df_v2.select("Height", "Height_cm").show(20, False)

display(fifa_df_v2.withColumn("Height",regexp_replace(col("Height"),"[0-9]","")).groupBy("Height").count().orderBy("count"))
'''

In [ ]:
# Converting the lbs to kg from weight column  and converting the weight column to int
from pyspark.sql.functions import regexp_replace, when, col, cast

# display how the data contains
display(fifa_df_v2.withColumn("Weight",regexp_replace("Weight","[0-9]","")).groupBy("Weight").count().orderBy("Weight"))


fifa_df_v2=fifa_df_v2.withColumn("Weight",
                                 when(col("Weight").like("%kg%"),                   # finding the replacing the kg to ""
                                      regexp_replace("Weight","kg","").cast("int")
                                      )                                      
                                 .when(col("Weight").like("%lbs%"),                  # finding the lbs and replacing the lbs to "" and converting the lbs to kg and casting to int
                                       regexp_replace("Weight","lbs","").cast("int")*0.453592                                 
                                       ).otherwise(None)
                                 ).withColumn("Weight",col("Weight").cast("int"))

display(fifa_df_v2.select("Weight"))

display(fifa_df_v2.withColumn("Weight",regexp_replace("Weight","[0-9]","")).groupBy("Weight").count().orderBy("Weight"))

In [ ]:
# REnaming the Columns to Hight to Height_in_cm and Weight to Weight_in_kg
fifa_df_v2=fifa_df_v2.withColumnRenamed("Height","Height_in_cm").withColumnRenamed("Weight","Weight_in_kg")
fifa_df_v2.printSchema()

In [ ]:
# converting "Value","Wage","Release Clause to int and 
# renameing into "Values in Euro Million", "Wage in Euros", "Release Clause in Euro Million"

from pyspark.sql.types import DoubleType
import re
def replacingValueWageRelaeaseClause(x):
    if x is None:
        return None
    num=str(x)
    num=re.sub(r"[^0-9.]","",num)
    if "K" in x:
        return float(num)*1000
    x=float(num)
    
    return x

replaceingValueWageRelaeaseClause_=udf(replacingValueWageRelaeaseClause,StringType())

fifa_df_v2=fifa_df_v2.withColumn("Value",replaceingValueWageRelaeaseClause_(col("Value"))).withColumn("Wage",replaceingValueWageRelaeaseClause_(col("Wage"))).withColumn("Release Clause",replaceingValueWageRelaeaseClause_(col("Release Clause"))).withColumnRenamed("Value","Values in Euro Million").withColumnRenamed("Wage","Wage in Euros").withColumnRenamed("Release Clause","Release Clause in Euro Million")
fifa_df_v2.printSchema()

In [ ]:
# W/F, SM, IR  replacing the star an converting to int 

fifa_df_v2=fifa_df_v2.withColumn("W/F",regexp_replace(col("W/F"),"★","")).withColumn("SM",regexp_replace(col("SM"),"★","")).withColumn("IR",regexp_replace(col("IR"),"★","")).withColumn("W/F",col("W/F").cast("int")).withColumn("SM",col("SM").cast("int")).withColumn("IR",col("IR").cast("int"))
# Replacing the columns to int  
fifa_df_v2=fifa_df_v2.withColumn("Crossing",col("Crossing").cast("int"))

In [ ]:

#display(fifa_df_v2.withColumn("Hits",regexp_replace("Hits","[0-9]","")).groupBy("Hits").count().orderBy("Hits"))


def hitsData(x):
    if x is None:
        return 0
    if ".K" in x:
        return float(x[:-2])*1000
    if "K" in x:
        return float(x[:-1])*1000
    
    return x   
    

hitsdatareplace=udf(hitsData,StringType())
fifa_df_v2=fifa_df_v2.withColumn("Hits",hitsdatareplace(col("Hits")))#when(col("Hits").like("%K%"),regexp_replace("Hits","[^0-9]","")*1000).otherwise("0"))

#display(fifa_df_v2.withColumn("Hits",regexp_replace("Hits","[0-9]","")).groupBy("Hits").count().orderBy("Hits"))

display(fifa_df_v2.select("Hits"))

In [ ]:
display(fifa_df_v2)